In [1]:
import re
import numpy as np
import pprint as pp

def tokenize(text):
    # obtains tokens with a least 1 alphabet
    pattern = re.compile(r'[A-Za-z]+[\w^\']*|[\w^\']*[A-Za-z]+[\w^\']*')
    return pattern.findall(text.lower())  

In [2]:
text = "natural language processing and machine learning is fun and exciting"
tokens = tokenize(text)
#pp.pprint(tokens)

In [3]:
wordCounts= {}
for eachWord in tokens:
    if eachWord in wordCounts.keys():
        wordCounts[eachWord] += 1
    else:
        wordCounts[eachWord] = 1
#wordCounts

In [4]:
word2index = {}
word2index = dict ((word, i) for i, word in enumerate(wordCounts.keys()))
# word2index 

Need to generate target word and context word based on window size. Each of these words are to be represented as one hot vectors. One hot vector is of dimension = (# of unique tokens in text) and has 1 in position of word under consideration.

In [5]:
params = {
    'window':3,
    'embed_dim':10,
    'N':len(tokens), #number of tokens
    'V':len(word2index), #number of unique words in the vocab
    'Ed':10,
    'Lr':0.1,
    'epochs':10000
}

In [6]:
def getOneHotVector(word, word2index):
    oneHot = np.zeros(len(word2index))
    wordInd = word2index[word]
    oneHot[wordInd] = 1
    return oneHot
a = getOneHotVector('language', word2index)
a.shape

(9,)

In [7]:
w = params['window']
N = params['N']
training_data = []
for i in range(N):
    w_target = getOneHotVector(tokens[i], word2index)
    contextIndices = list(range(max(0, i-w), i)) + list(range(i+1, min(i+w+1, N)))
    w_context = []
    for eachContextIndex in contextIndices:
        w_context.append(getOneHotVector(tokens[eachContextIndex], word2index))
    training_data.append([w_target, w_context])

In [8]:
training_data

[[array([1., 0., 0., 0., 0., 0., 0., 0., 0.]),
  [array([0., 1., 0., 0., 0., 0., 0., 0., 0.]),
   array([0., 0., 1., 0., 0., 0., 0., 0., 0.]),
   array([0., 0., 0., 1., 0., 0., 0., 0., 0.])]],
 [array([0., 1., 0., 0., 0., 0., 0., 0., 0.]),
  [array([1., 0., 0., 0., 0., 0., 0., 0., 0.]),
   array([0., 0., 1., 0., 0., 0., 0., 0., 0.]),
   array([0., 0., 0., 1., 0., 0., 0., 0., 0.]),
   array([0., 0., 0., 0., 1., 0., 0., 0., 0.])]],
 [array([0., 0., 1., 0., 0., 0., 0., 0., 0.]),
  [array([1., 0., 0., 0., 0., 0., 0., 0., 0.]),
   array([0., 1., 0., 0., 0., 0., 0., 0., 0.]),
   array([0., 0., 0., 1., 0., 0., 0., 0., 0.]),
   array([0., 0., 0., 0., 1., 0., 0., 0., 0.]),
   array([0., 0., 0., 0., 0., 1., 0., 0., 0.])]],
 [array([0., 0., 0., 1., 0., 0., 0., 0., 0.]),
  [array([1., 0., 0., 0., 0., 0., 0., 0., 0.]),
   array([0., 1., 0., 0., 0., 0., 0., 0., 0.]),
   array([0., 0., 1., 0., 0., 0., 0., 0., 0.]),
   array([0., 0., 0., 0., 1., 0., 0., 0., 0.]),
   array([0., 0., 0., 0., 0., 1., 0., 

##### Training data is ready. Build the network structure. Ignoring the bias terms, I need:
    - Hidden layer weights
    - Output layer weights
    
##### What does the NN do?
    - Since this is a skipgram model, it predicts the context words given a center word.

##### What is the output of NN?
    - Output is the probability distribution over all the unique words such that the context words will have high probability than non context words.
    
##### What is the shape of weight matrices of hidden and output layer?
    - Embedding dimensions is provided in the parameters as Ed
    - Input Vector = [V,1]
    - Wh = [V, Ed]
    - Wo = [Ed, V]
    - Output Vector = [V, Ed]
    
##### Why dont we have an activation function in the hidden layer?

In [9]:
params['Wh'] = np.random.uniform(-1,1,(params['V'], params['Ed']))
print('Hidden layer weights Wh:', params['Wh'].shape)
params['Wo'] = np.random.uniform(-1,1,(params['Ed'], params['V']))
print('Output layer weights Wo:', params['Wo'].shape)
params['Wh'] 
params['Wo']

Hidden layer weights Wh: (9, 10)
Output layer weights Wo: (10, 9)


array([[-0.0676114 ,  0.68857534, -0.8924202 , -0.82839605,  0.68152572,
         0.00257424, -0.36013001, -0.46975061, -0.86721951],
       [ 0.15165494,  0.02267613, -0.33423338, -0.39161091,  0.77521506,
         0.97980448, -0.83557118,  0.5730265 ,  0.00320183],
       [ 0.07218461,  0.66497996, -0.42133111, -0.22804522, -0.6280436 ,
         0.62114312,  0.82116103,  0.16125765, -0.22479689],
       [-0.15132406,  0.27727443,  0.84469449,  0.63922409,  0.25601281,
        -0.52426068, -0.53823786, -0.74410497, -0.68221234],
       [-0.23204985, -0.32141216, -0.47578448, -0.23548055,  0.88689939,
        -0.29940789, -0.72601733,  0.10321062,  0.3667019 ],
       [ 0.11485066, -0.95554856,  0.10859713,  0.77782798, -0.70649008,
        -0.78581115,  0.67257709,  0.93467395, -0.1365687 ],
       [ 0.30212152,  0.87315748, -0.10217679, -0.07413153,  0.39726299,
        -0.4991837 , -0.99824058, -0.89042076,  0.16505447],
       [-0.03337596, -0.16718992,  0.92996316,  0.45157786, -0

In [10]:
def forwardPass(inputVector, Wh, Wo):
    h = np.dot(Wh.T, inputVector)
#     print('Wh Shape:',Wh.shape)
#     print('Input Vector shape:', inputVector.shape)
#     print('input vector:',inputVector)
    u = np.dot(Wo.T, h)
    o = softmax(u)
#     print(o)
    return h,u,o

In [11]:
def softmax(x):
    sm = np.exp(x)/np.exp(x).sum()
    return sm

In [12]:
def backProp(error, h, target):
    dl_dw2 = np.outer(h, error)
    dl_dw1 = np.outer(target, np.dot(params['Wo'], error.T))
    # Update weights
    params['Wh'] = params['Wh'] - (params['Lr'] * dl_dw1)
    params['Wo'] = params['Wo'] - (params['Lr'] * dl_dw2)

In [13]:
inits = params['Wh']
inits[0]

array([-0.5572125 , -0.64269345,  0.29576338,  0.4178194 ,  0.89434003,
       -0.15172811,  0.39749312,  0.8132315 ,  0.55131691,  0.99414663])

In [14]:
for i in range(params['epochs']):
    loss = 0
    for target, context in training_data:
        h, u, o = forwardPass(target, params['Wh'], params['Wo'])
        error = 0
#         error = np.sum([np.subtract(o, ec) for ec in context ], axis=0)
        for eachContext in context:
            error += np.subtract(o, eachContext)
        backProp(error, h, target)
        loss += -np.sum([u[word.tolist().index(1)] for word in context]) + len(context) * np.log(np.sum(np.exp(u)))
    print('Epoch:', i, ' Loss:', loss)


Epoch: 0  Loss: 123.95218209300884
Epoch: 1  Loss: 102.18881159617877
Epoch: 2  Loss: 96.82119386481926
Epoch: 3  Loss: 93.6850546267322
Epoch: 4  Loss: 91.4799772830763
Epoch: 5  Loss: 89.76598852021593
Epoch: 6  Loss: 88.36967400584508
Epoch: 7  Loss: 87.20817002501639
Epoch: 8  Loss: 86.23031981161061
Epoch: 9  Loss: 85.40060008063207
Epoch: 10  Loss: 84.69388093013559
Epoch: 11  Loss: 84.09148964661871
Epoch: 12  Loss: 83.57818826158056
Epoch: 13  Loss: 83.140610922024
Epoch: 14  Loss: 82.76681114682732
Epoch: 15  Loss: 82.4462656915007
Epoch: 16  Loss: 82.1699254084988
Epoch: 17  Loss: 81.93018232560895
Epoch: 18  Loss: 81.7207545792371
Epoch: 19  Loss: 81.53652000088695
Epoch: 20  Loss: 81.37332962196245
Epoch: 21  Loss: 81.22782637566326
Epoch: 22  Loss: 81.09728569610506
Epoch: 23  Loss: 80.97948447739883
Epoch: 24  Loss: 80.87259770668766
Epoch: 25  Loss: 80.77511821401332
Epoch: 26  Loss: 80.68579448304068
Epoch: 27  Loss: 80.60358195774528
Epoch: 28  Loss: 80.5276045797643
E

Epoch: 313  Loss: 79.37893363924722
Epoch: 314  Loss: 79.29576109807437
Epoch: 315  Loss: 79.37423987588957
Epoch: 316  Loss: 79.29221378408414
Epoch: 317  Loss: 79.37003357591674
Epoch: 318  Loss: 79.29012548478462
Epoch: 319  Loss: 79.36712086707017
Epoch: 320  Loss: 79.29008039705202
Epoch: 321  Loss: 79.36638193217232
Epoch: 322  Loss: 79.29250647803838
Epoch: 323  Loss: 79.36854200422385
Epoch: 324  Loss: 79.29751826927725
Epoch: 325  Loss: 79.37400469155355
Epoch: 326  Loss: 79.3048105755461
Epoch: 327  Loss: 79.38298925256868
Epoch: 328  Loss: 79.31368494075289
Epoch: 329  Loss: 79.39613212700783
Epoch: 330  Loss: 79.3231651500673
Epoch: 331  Loss: 79.41514051367876
Epoch: 332  Loss: 79.33174390801398
Epoch: 333  Loss: 79.44129238390114
Epoch: 334  Loss: 79.33582101656886
Epoch: 335  Loss: 79.46810238311379
Epoch: 336  Loss: 79.32910972595872
Epoch: 337  Loss: 79.47572407081613
Epoch: 338  Loss: 79.31294147239622
Epoch: 339  Loss: 79.45093393434786
Epoch: 340  Loss: 79.309825232

Epoch: 563  Loss: 78.98004750994815
Epoch: 564  Loss: 79.08261238930989
Epoch: 565  Loss: 78.9764383531309
Epoch: 566  Loss: 79.013469385296
Epoch: 567  Loss: 78.87425349259226
Epoch: 568  Loss: 78.97739205617457
Epoch: 569  Loss: 78.94096365395689
Epoch: 570  Loss: 79.1174506919071
Epoch: 571  Loss: 79.34543115894442
Epoch: 572  Loss: 79.59292373068259
Epoch: 573  Loss: 79.55517233567213
Epoch: 574  Loss: 79.56923065611974
Epoch: 575  Loss: 79.43589298890197
Epoch: 576  Loss: 79.70103512585159
Epoch: 577  Loss: 80.07849635986544
Epoch: 578  Loss: 80.24572287071086
Epoch: 579  Loss: 80.40484663311969
Epoch: 580  Loss: 80.40284620983863
Epoch: 581  Loss: 79.8511666467761
Epoch: 582  Loss: 79.74817225265342
Epoch: 583  Loss: 79.80525897900128
Epoch: 584  Loss: 79.70969881705186
Epoch: 585  Loss: 79.78090137724627
Epoch: 586  Loss: 79.66336059474783
Epoch: 587  Loss: 79.63233301831073
Epoch: 588  Loss: 79.41157154906618
Epoch: 589  Loss: 79.37771303002424
Epoch: 590  Loss: 79.145850007351

Epoch: 898  Loss: 79.05949927799676
Epoch: 899  Loss: 79.16611062620534
Epoch: 900  Loss: 79.00734336272811
Epoch: 901  Loss: 79.09232182855544
Epoch: 902  Loss: 78.99779506906398
Epoch: 903  Loss: 79.23587148904642
Epoch: 904  Loss: 79.13598777078833
Epoch: 905  Loss: 79.37610076981886
Epoch: 906  Loss: 79.34624485280362
Epoch: 907  Loss: 79.60198255327074
Epoch: 908  Loss: 79.49533323236044
Epoch: 909  Loss: 79.57664611704928
Epoch: 910  Loss: 79.55633399230727
Epoch: 911  Loss: 79.70361538871765
Epoch: 912  Loss: 79.83974814410612
Epoch: 913  Loss: 79.90593900379288
Epoch: 914  Loss: 80.00442583881629
Epoch: 915  Loss: 79.98416829543169
Epoch: 916  Loss: 79.60197257196509
Epoch: 917  Loss: 79.56638049259698
Epoch: 918  Loss: 79.50950768176807
Epoch: 919  Loss: 79.52804037879744
Epoch: 920  Loss: 79.4779856570113
Epoch: 921  Loss: 79.55407360009886
Epoch: 922  Loss: 79.47457800125157
Epoch: 923  Loss: 79.43544490043753
Epoch: 924  Loss: 79.28606333669057
Epoch: 925  Loss: 79.14589474

Epoch: 1164  Loss: 79.10292529829506
Epoch: 1165  Loss: 79.14277517566074
Epoch: 1166  Loss: 79.26041561029452
Epoch: 1167  Loss: 79.40257720542317
Epoch: 1168  Loss: 79.57990354237384
Epoch: 1169  Loss: 79.74497525480932
Epoch: 1170  Loss: 79.8780420381362
Epoch: 1171  Loss: 79.87094470202867
Epoch: 1172  Loss: 79.87151612674393
Epoch: 1173  Loss: 79.68443617133806
Epoch: 1174  Loss: 79.68450201517774
Epoch: 1175  Loss: 79.50662527918375
Epoch: 1176  Loss: 79.5325201304857
Epoch: 1177  Loss: 79.52431980269311
Epoch: 1178  Loss: 79.52171350968314
Epoch: 1179  Loss: 79.56421810636633
Epoch: 1180  Loss: 79.57304605909864
Epoch: 1181  Loss: 79.6170974032275
Epoch: 1182  Loss: 79.53821188272632
Epoch: 1183  Loss: 79.64658957411952
Epoch: 1184  Loss: 79.35870991698302
Epoch: 1185  Loss: 79.50465359511554
Epoch: 1186  Loss: 79.17382223784705
Epoch: 1187  Loss: 79.28733520286613
Epoch: 1188  Loss: 79.1061963198309
Epoch: 1189  Loss: 79.28107999114677
Epoch: 1190  Loss: 79.16189608121068
Epoch

...

Epoch: 8739  Loss: 79.93869563996887
Epoch: 8740  Loss: 79.90332885355612
Epoch: 8741  Loss: 79.7024741259032
Epoch: 8742  Loss: 79.72359772685559
Epoch: 8743  Loss: 79.5613073133486
Epoch: 8744  Loss: 79.48557726144546
Epoch: 8745  Loss: 79.32009783217212
Epoch: 8746  Loss: 79.40130746994018
Epoch: 8747  Loss: 79.4551076377455
Epoch: 8748  Loss: 79.7050326777355
Epoch: 8749  Loss: 79.83641036823843
Epoch: 8750  Loss: 79.8036758317203
Epoch: 8751  Loss: 79.7715788662097
Epoch: 8752  Loss: 80.03421093889635
Epoch: 8753  Loss: 79.91240560823476
Epoch: 8754  Loss: 79.93607938076252
Epoch: 8755  Loss: 79.68477692351861
Epoch: 8756  Loss: 79.77934426816692
Epoch: 8757  Loss: 79.6636896109989
Epoch: 8758  Loss: 79.50518511330125
Epoch: 8759  Loss: 79.34325677955803
Epoch: 8760  Loss: 79.4441904703135
Epoch: 8761  Loss: 79.47473394669126
Epoch: 8762  Loss: 79.71839833051271
Epoch: 8763  Loss: 79.87770543086447
Epoch: 8764  Loss: 79.80731882816693
Epoch: 8765  Loss: 79.79072638158374
Epoch: 87

Epoch: 9069  Loss: 79.71720151182832
Epoch: 9070  Loss: 79.70670807012517
Epoch: 9071  Loss: 79.58584113744274
Epoch: 9072  Loss: 79.4823216732155
Epoch: 9073  Loss: 79.29485421454297
Epoch: 9074  Loss: 79.40807720697279
Epoch: 9075  Loss: 79.4226531112786
Epoch: 9076  Loss: 79.53667683649793
Epoch: 9077  Loss: 79.64969596573425
Epoch: 9078  Loss: 79.73692412473446
Epoch: 9079  Loss: 79.72803869867565
Epoch: 9080  Loss: 79.89586106061026
Epoch: 9081  Loss: 79.843579010549
Epoch: 9082  Loss: 80.0341352691588
Epoch: 9083  Loss: 79.80621022853796
Epoch: 9084  Loss: 79.7145016669123
Epoch: 9085  Loss: 79.59418036330504
Epoch: 9086  Loss: 79.57374111824456
Epoch: 9087  Loss: 79.42924655735945
Epoch: 9088  Loss: 79.46124131119339
Epoch: 9089  Loss: 79.44826162353854
Epoch: 9090  Loss: 79.64829659167664
Epoch: 9091  Loss: 79.77656297139107
Epoch: 9092  Loss: 79.89059764175826
Epoch: 9093  Loss: 79.90845603325118
Epoch: 9094  Loss: 79.9828160094516
Epoch: 9095  Loss: 79.92914251242497
Epoch: 9

Epoch: 9352  Loss: 80.08748803582668
Epoch: 9353  Loss: 79.9050357841513
Epoch: 9354  Loss: 79.87353595202343
Epoch: 9355  Loss: 79.60852846737099
Epoch: 9356  Loss: 79.6575425863916
Epoch: 9357  Loss: 79.52166182910653
Epoch: 9358  Loss: 79.47089596506838
Epoch: 9359  Loss: 79.30656714953577
Epoch: 9360  Loss: 79.38805040510374
Epoch: 9361  Loss: 79.43026175076426
Epoch: 9362  Loss: 79.658293371953
Epoch: 9363  Loss: 79.81776716780556
Epoch: 9364  Loss: 79.84610180923542
Epoch: 9365  Loss: 79.83672874147143
Epoch: 9366  Loss: 80.1213338923749
Epoch: 9367  Loss: 80.00699420094739
Epoch: 9368  Loss: 80.08110043084628
Epoch: 9369  Loss: 79.7940339494446
Epoch: 9370  Loss: 79.75203498481571
Epoch: 9371  Loss: 79.63425834243803
Epoch: 9372  Loss: 79.5648013366861
Epoch: 9373  Loss: 79.41471286294693
Epoch: 9374  Loss: 79.49048837573145
Epoch: 9375  Loss: 79.50731272649719
Epoch: 9376  Loss: 79.61816913437245
Epoch: 9377  Loss: 79.73755455534007
Epoch: 9378  Loss: 79.84632363573174
Epoch: 9

Epoch: 9686  Loss: 79.60454003379971
Epoch: 9687  Loss: 79.41570951340003
Epoch: 9688  Loss: 79.41264184187663
Epoch: 9689  Loss: 79.34739650847823
Epoch: 9690  Loss: 79.52200236354417
Epoch: 9691  Loss: 79.70471266705154
Epoch: 9692  Loss: 79.96232483549532
Epoch: 9693  Loss: 80.00771596631019
Epoch: 9694  Loss: 79.98117293451088
Epoch: 9695  Loss: 80.0534176983233
Epoch: 9696  Loss: 80.34125950450652
Epoch: 9697  Loss: 79.97920443312341
Epoch: 9698  Loss: 79.91567643041176
Epoch: 9699  Loss: 79.81629961120952
Epoch: 9700  Loss: 79.65466208519221
Epoch: 9701  Loss: 79.39609133452166
Epoch: 9702  Loss: 79.47478597480789
Epoch: 9703  Loss: 79.42720377326529
Epoch: 9704  Loss: 79.46518702539133
Epoch: 9705  Loss: 79.61194857692752
Epoch: 9706  Loss: 80.0019261885566
Epoch: 9707  Loss: 80.14861552173565
Epoch: 9708  Loss: 80.02310102673923
Epoch: 9709  Loss: 80.1092835584145
Epoch: 9710  Loss: 80.32009327145454
Epoch: 9711  Loss: 79.99089957176255
Epoch: 9712  Loss: 79.82038021979497
Epoc

Epoch: 9971  Loss: 79.95691803503972
Epoch: 9972  Loss: 80.13088276377476
Epoch: 9973  Loss: 79.94781474294322
Epoch: 9974  Loss: 79.96642520568555
Epoch: 9975  Loss: 79.71182121281818
Epoch: 9976  Loss: 79.78780252061304
Epoch: 9977  Loss: 79.70436326518887
Epoch: 9978  Loss: 79.6809378957204
Epoch: 9979  Loss: 79.46194767517063
Epoch: 9980  Loss: 79.48472483025971
Epoch: 9981  Loss: 79.47617950619005
Epoch: 9982  Loss: 79.645987373569
Epoch: 9983  Loss: 79.63298518760266
Epoch: 9984  Loss: 79.90270537554497
Epoch: 9985  Loss: 79.87254994698361
Epoch: 9986  Loss: 80.03009983891792
Epoch: 9987  Loss: 79.96334654328699
Epoch: 9988  Loss: 80.16322814624857
Epoch: 9989  Loss: 79.88480116251682
Epoch: 9990  Loss: 79.90905654150225
Epoch: 9991  Loss: 79.7987826220509
Epoch: 9992  Loss: 79.59334372571956
Epoch: 9993  Loss: 79.37707556256838
Epoch: 9994  Loss: 79.56808028279485
Epoch: 9995  Loss: 79.53333123385025
Epoch: 9996  Loss: 79.66195458549576
Epoch: 9997  Loss: 79.61249071635427
Epoch

In [18]:
emb = params['Wh']
emb[0]
print(emb.shape)

(9, 10)
